In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

# Query both residential land use and residential buildings for the Philippines
residential = ox.features_from_place("Philippines", tags={'landuse': 'residential'})
residential_buildings = ox.features_from_place("Philippines", tags={'building': 'residential'})

In [11]:
target_crs = "EPSG:32651"

res_bldgs = residential_buildings['geometry']['way'].to_crs(target_crs)
res_areas = residential['geometry']['way'].to_crs(target_crs)

# Concatenate and remove duplicate geometries
residential = gpd.GeoDataFrame(
    pd.concat([res_areas, res_bldgs], ignore_index=True),
    crs=target_crs
).drop_duplicates(subset='geometry')

In [13]:
# Optionally, save to a GeoJSON or any other format
residential.to_file("../01_processed_data/residential_areas.geojson", driver="GeoJSON")

In [17]:
import geopandas as gpd
residential = gpd.read_file("../01_processed_data/residential_areas.geojson")
nearest_geom = residential.geometry.union_all()  # combines all polygons

In [3]:
import geopandas as gpd

residential = gpd.read_file("../01_processed_data/residential_areas.geojson")
residential

,geometry
0,"POLYGON ((287268.972 1610211.895, 287247.366 1..."
1,"POLYGON ((288533.551 1610975.165, 288532.031 1..."
2,"POLYGON ((286861.693 1596040.753, 287119.755 1..."
3,"POLYGON ((287355.511 1596799.103, 287633.221 1..."
4,"POLYGON ((293084.285 1616170.368, 293083.35 16..."
...,...
190580,"POLYGON ((335844.334 1409388.816, 335854.381 1..."
190581,"POLYGON ((297599.223 1617832.323, 297607.88 16..."
190582,"POLYGON ((297580.901 1617829.299, 297587.775 1..."
190583,"POLYGON ((297589.549 1617831.341, 297597.372 1..."


In [ ]:
# Split residential gdf into two separate GeoDataFrames
residential_1 = residential.iloc[:len(residential)//2]
residential_2 = residential.iloc[len(residential)//2:]
# Save the two GeoDataFrames to separate files
residential_1.to_file("../01_processed_data/residential_areas_part1.geojson", driver="GeoJSON")
residential_2.to_file("../01_processed_data/residential_areas_part2.geojson", driver="GeoJSON")
# Read the two GeoDataFrames from the files
residential_1 = gpd.read_file("../01_processed_data/residential_areas_part1.geojson")
residential_2 = gpd.read_file("../01_processed_data/residential_areas_part2.geojson")

In [8]:
import pandas as pd
# Combine the two GeoDataFrames into one
residential_combined = pd.concat([residential_1, residential_2], ignore_index=True)
residential_combined

,geometry
0,"POLYGON ((287268.972 1610211.895, 287247.366 1..."
1,"POLYGON ((288533.551 1610975.165, 288532.031 1..."
2,"POLYGON ((286861.693 1596040.753, 287119.755 1..."
3,"POLYGON ((287355.511 1596799.103, 287633.221 1..."
4,"POLYGON ((293084.285 1616170.368, 293083.35 16..."
...,...
190580,"POLYGON ((335844.334 1409388.816, 335854.381 1..."
190581,"POLYGON ((297599.223 1617832.323, 297607.88 16..."
190582,"POLYGON ((297580.901 1617829.299, 297587.775 1..."
190583,"POLYGON ((297589.549 1617831.341, 297597.372 1..."


In [4]:
# Wrap the GeometryCollection or MultiPolygon in a GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=[nearest_geom], crs="EPSG:32651")

# Save to file (e.g., GeoJSON)
gdf.to_file("../01_processed_data/residential_areas_union.geojson", driver="GeoJSON")

In [5]:
gdf

,geometry
0,GEOMETRYCOLLECTION (POLYGON ((299825.219 16190...


In [44]:
from shapely.geometry import Point
import pandas as pd

coordinates = [
    (13.4125, 122.5623),
    (10.7103, 122.5621),
    (16.4023, 120.596),
    (15.489, 120.9739),
    (14.5995, 120.9842),
    (8.051, 124.92),
    (6.9214, 122.079),
    (11.0046, 124.6093),
    (13.54, 122.5623),
    (17.6133, 121.7269),
]

geom = gdf.geometry.iloc[0]

points_df = pd.DataFrame(coordinates, columns=["lat", "lon"])
points_geometry = [Point(lon, lat) for lat, lon in coordinates]  # Note: (lon, lat) for Point
points = gpd.GeoDataFrame(points_df, geometry=points_geometry, crs="EPSG:4326")
points = points.to_crs("EPSG:32651")


points['min_distance_to_res'] = points.geometry.apply(lambda x: x.distance(geom))
points

,lat,lon,geometry,min_distance_to_res
0,13.4125,122.5623,POINT (452614.416 1482795.258),199.151270
1,10.7103,122.5621,POINT (452115.58 1183981.478),263.565808
2,16.4023,120.5960,POINT (243249.465 1814958.033),193.691104
3,15.4890,120.9739,POINT (282642.915 1713439.358),959.256187
4,14.5995,120.9842,POINT (282848.981 1614992.835),8.100706
5,8.0510,124.9200,POINT (711594.369 890432.662),704.416245
6,6.9214,122.0790,POINT (398253.394 765158.476),841.151586
7,11.0046,124.6093,POINT (675824.968 1216959.499),613.646517
8,13.5400,122.5623,POINT (452639.519 1496895.897),79.169581
9,17.6133,121.7269,POINT (364926.455 1947857.541),223.662385


In [24]:
points.geometry

0    POINT (122.5623 13.4125)
1    POINT (122.5621 10.7103)
2     POINT (120.596 16.4023)
3     POINT (120.9739 15.489)
4    POINT (120.9842 14.5995)
5        POINT (124.92 8.051)
6      POINT (122.079 6.9214)
7    POINT (124.6093 11.0046)
8      POINT (122.5623 13.54)
9    POINT (121.7269 17.6133)
Name: geometry, dtype: geometry

In [25]:
residential

,geometry
0,"POLYGON ((287268.972 1610211.895, 287247.366 1..."
1,"POLYGON ((288533.551 1610975.165, 288532.031 1..."
2,"POLYGON ((286861.693 1596040.753, 287119.755 1..."
3,"POLYGON ((287355.511 1596799.103, 287633.221 1..."
4,"POLYGON ((293084.285 1616170.368, 293083.35 16..."
...,...
190580,"POLYGON ((335844.334 1409388.816, 335854.381 1..."
190581,"POLYGON ((297599.223 1617832.323, 297607.88 16..."
190582,"POLYGON ((297580.901 1617829.299, 297587.775 1..."
190583,"POLYGON ((297589.549 1617831.341, 297597.372 1..."


In [26]:
residential = residential[residential.geometry.notnull()]
residential

,geometry
0,"POLYGON ((287268.972 1610211.895, 287247.366 1..."
1,"POLYGON ((288533.551 1610975.165, 288532.031 1..."
2,"POLYGON ((286861.693 1596040.753, 287119.755 1..."
3,"POLYGON ((287355.511 1596799.103, 287633.221 1..."
4,"POLYGON ((293084.285 1616170.368, 293083.35 16..."
...,...
190580,"POLYGON ((335844.334 1409388.816, 335854.381 1..."
190581,"POLYGON ((297599.223 1617832.323, 297607.88 16..."
190582,"POLYGON ((297580.901 1617829.299, 297587.775 1..."
190583,"POLYGON ((297589.549 1617831.341, 297597.372 1..."


In [38]:
res_geom_list = list(residential.geometry.values)  # ensure it's a plain list of geometries
res_geom_list

[<POLYGON ((287268.972 1610211.895, 287247.366 1610169.056, 287305.602 161014...>,
 <POLYGON ((288533.551 1610975.165, 288532.031 1610915.12, 288574.255 1610914...>,
 <POLYGON ((286861.693 1596040.753, 287119.755 1595958.347, 287146.693 159594...>,
 <POLYGON ((287355.511 1596799.103, 287633.221 1596800.394, 287630.961 159638...>,
 <POLYGON ((293084.285 1616170.368, 293083.35 1616171.925, 293055.502 1616199...>,
 <POLYGON ((298242.921 1578901.357, 298248.281 1578776.261, 298226.514 157877...>,
 <POLYGON ((319781.949 1556274.51, 319604.736 1556221.142, 319643.179 1556134...>,
 <POLYGON ((319676.623 1555321.979, 319594.036 1555272.554, 319565.99 1555314...>,
 <POLYGON ((306072.477 1567445.867, 306020.97 1567458.799, 305941.218 1567477...>,
 <POLYGON ((523926.533 1506917.058, 523944.444 1507071.66, 523936.933 1507101...>,
 <POLYGON ((285802.788 1611401.844, 285881.555 1611397.814, 285900.36 1611346...>,
 <POLYGON ((283609.805 1608890.884, 283860.295 1608955.711, 283945.896 160856...>,
 <PO

In [43]:
from shapely.strtree import STRtree
import numpy as np

# 1. Create a spatial index from residential geometries
res_geom_list = list(residential.geometry.values)  # ensure it's a plain list of geometries
res_tree = STRtree(res_geom_list)

# 2. For each site, find the nearest residential polygon and compute distance
def nearest_distance(site_geom):
    if site_geom is None or site_geom.is_empty:
        return np.nan
    nearest_idx = res_tree.nearest(site_geom)
    nearest_geom = res_tree.geometries.take(nearest_idx)

    return site_geom.distance(nearest_geom)

# Apply to each site geometry
points['min_distance_to_res'] = points.geometry.apply(nearest_distance)
points

,lat,lon,geometry,min_distance_to_res
0,13.4125,122.5623,POINT (452614.416 1482795.258),199.151270
1,10.7103,122.5621,POINT (452115.58 1183981.478),263.565808
2,16.4023,120.5960,POINT (243249.465 1814958.033),193.691104
3,15.4890,120.9739,POINT (282642.915 1713439.358),959.256187
4,14.5995,120.9842,POINT (282848.981 1614992.835),8.100706
5,8.0510,124.9200,POINT (711594.369 890432.662),704.416245
6,6.9214,122.0790,POINT (398253.394 765158.476),841.151586
7,11.0046,124.6093,POINT (675824.968 1216959.499),613.646517
8,13.5400,122.5623,POINT (452639.519 1496895.897),79.169581
9,17.6133,121.7269,POINT (364926.455 1947857.541),223.662385
